# Experiment 1 Notebook

In this experiment we build a machine learning model around the diabetes_v1 csv file. In this version of the dataset we have eliminated the records with missing data as well as the outliers reducing the amount of records by a half.

In [1]:
import pandas as pd